In [ ]:
!pip install python-docx 

In [ ]:
document = file.docx # finds reddit.com urls in a word document. Must look like this: (reddit.com/r/subform/)

In [ ]:
# convert reddit url to RIS file
import json
import urllib
from urllib.request import urlopen
from datetime import datetime
from time import sleep
from tqdm.auto import tqdm
from docx import Document
import re

document = Document(document)
urls = []
for paragraph in document.paragraphs:
  text=re.findall('\((.*?)\)',paragraph.text)
  for word in text:
    if "reddit.com/r/" in word:
      urls.append(word)
print(urls)


def url_to_api(url):
  url = url.split("reddit.com/r/")[1]
  parts = url.split('/') #0 subreddit # 1 comments #2 uuid #3 title
  return f"https://api.reddit.com/api/info/?id=t3_{parts[2]}&raw_json=1"

In [ ]:
def api_to_json(api):
  response = urlopen(api)
  return json.loads(response.read())

In [ ]:
def json_to_ris(json):
  json = json_data['data']['children'][0]['data']
  author = "AU "+json['author'] # AU
  date = "PY "+datetime.utcfromtimestamp(json['created']).strftime('%Y') # DA/Y1
  accessed = "Y2 " + "2022-02-02" # Y2
  title = "TI "+json['title'] # TI
  site = "UR "+json['url'] # UR
  abstract = json['selftext'].replace("\n","")
  abstract = "AB "+(abstract if len(abstract)<=200 else abstract[:200]+"...") # AB

  return f"TY BLOG\nJA reddit.com/r/{json['subreddit']}\n{author}\n{date}\n{title}\n{accessed}\n{site}\n{abstract}\nER\n"


In [ ]:
json_data['data']['children'][0]['data']

In [ ]:
ris="\n"
#for i,url2 in enumerate(urls):
for url in tqdm(urls):
  print(url)
  i=0
  while True:
    try:
      api = url_to_api(url)
      json_data = api_to_json(api)
      ris+=json_to_ris(json_data)
      break
    except urllib.error.HTTPError:
      print("Rate limited. Sleeping one minute")
      for i in tqdm(range(60)):
        sleep(1)
      continue

print(ris)
with open('reddit.ris', 'w') as f:
    f.write(ris)



  0%|          | 0/4 [00:00<?, ?it/s]

reddit.com/r/BPD/comments/7x3jv4/i_feel_like_the_universe_is_asking_me_to_die/
reddit.com/r/BPD/comments/akhm9o/paranoia/
reddit.com/r/BPD/comments/9o804o/just_looking_for_some_advice_with_paranoia/
Rate limited. Sleeping one minute


  0%|          | 0/60 [00:00<?, ?it/s]

reddit.com/r/BPD/comments/4z7bks/in_a_new_relationship/

TY BLOG
JA reddit.com/r/BPD
AU GoneUnsane
PY 2018
TI I feel like the universe is asking me to die
Y2 2022-02-02
UR https://www.reddit.com/r/BPD/comments/7x3jv4/i_feel_like_the_universe_is_asking_me_to_die/
AB I feel like I have always had BPD, I've still got vivid memories of elementary school classes, and recess. Neither were ever enjoyable to me, I'd spend most of the time stargazing why life happen to m...
ER
TY BLOG
JA reddit.com/r/BPD
AU wendypan105
PY 2019
TI Paranoia
Y2 2022-02-02
UR https://www.reddit.com/r/BPD/comments/akhm9o/paranoia/
AB I've been with my partner 7 nearly 8 months and while he hasn't  shown any reason to distrust him  I am  just so paranoid I am scared that in the future he will get a job and there will be other women...
ER
TY BLOG
JA reddit.com/r/BPD
AU everydayimfumbling
PY 2018
TI Just looking for some advice with paranoia
Y2 2022-02-02
UR https://www.reddit.com/r/BPD/comments/9o804o/just_looking_for